<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/resources/cloud_classification_ccsn_ViT_kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloud Image Classification using Vision Transformers with Stratified K-Fold Cross Validation


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!cp -r "/content/drive/My Drive/img_satelite/classificacao/CCSN/" "ccsn_full"

In [ ]:
#!pip install timm==0.3.2
#!pip install -U transformers
#!pip install -U scikit-learn
!pip install focal-loss
#!sudo apt -qq install git-lfs
#!pip install tensorflow_addons

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,GlobalAveragePooling2D ,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout,Input,Reshape
from keras.models import Sequential,Model
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, UnidentifiedImageError
from pathlib import Path
import dill
import pandas as pd
from huggingface_hub import HfApi, Repository
from transformers import ViTFeatureExtractor,TFViTModel,TFDeiTModel,ViTForImageClassification, ViTConfig,TFViTForImageClassification,DeiTForImageClassification,BeitForImageClassification,DeiTFeatureExtractor,BeitFeatureExtractor
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import SGD,Adam
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
import tensorflow_addons as tfa
from focal_loss import SparseCategoricalFocalLoss
from keras import backend as K
K.set_image_data_format('channels_first')

In [ ]:
def categorical_focal_loss(alpha, gamma=2.):

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed


In [ ]:
path_data = 'ccsn_full'

modelo

In [ ]:
def classification_model(sel_model,learning_rate,activation_f,num_labels,prob,number_of_neurons):

    if sel_model == 'vit':
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
        #model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        model = TFViTForImageClassification.from_pretrained(
            'google/vit-base-patch16-224-in21k',
            num_labels=num_labels)
    else:# sel_model == 'deit':
        feature_extractor = DeiTFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-224')
        model = TFDeiTModel.from_pretrained("facebook/deit-base-distilled-patch16-224")
    #else:
        #feature_extractor = BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-224')
        #model = TFBeitModel.from_pretrained('google/vit-base-patch16-224-in21k')
    
    #model.compile(optimizer="adam")
    
    return feature_extractor,model

Define the image generator objects

In [ ]:
import os
list_subfolders = os.listdir(path_data)
    
list_subfolders.sort()

dataset_dict = {}

dataset_dict['filename'] = []
dataset_dict['label'] = []

for folder in list_subfolders:

    list_images_path = os.listdir(path_data+"/"+folder)
    
    list_images_path.sort()

    for image_name in list_images_path:

        dataset_dict['filename'].append(folder+"/"+image_name)

        dataset_dict['label'].append(folder)

In [ ]:
df = pd.DataFrame.from_dict(dataset_dict)
df

,filename,label
0,Ac/Ac-N001.jpg,Ac
1,Ac/Ac-N002.jpg,Ac
2,Ac/Ac-N003.jpg,Ac
3,Ac/Ac-N004.jpg,Ac
4,Ac/Ac-N005.jpg,Ac
...,...,...
2538,St/St-N198.jpg,St
2539,St/St-N199.jpg,St
2540,St/St-N200.jpg,St
2541,St/St-N201.jpg,St


In [ ]:
def returnModel(num_labels,activation_f,batch_size):

    configuration = ViTConfig()

    # Initializing a model from the vit-base-patch16-224 style configuration
    model = TFViTModel(configuration)
    
    base_model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

    # Flipping and rotating images
    #data_augmentation = Sequential(
    #    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
    #im.transpose(2,0,1)
    #)

    # Freeze base model
    base_model.trainable = False

    # Create new model
    inputs = Input(shape = (3,224, 224))
    #x = data_augmentation(inputs)   # apply data augmentation
    x = base_model(inputs, training=False)[0]
    x = Flatten()(x)
    outputs = Dense(num_labels, activation=activation_f)(x)
    model_vit = Model(inputs, outputs)
    return model_vit

In [ ]:
skf_outer = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

outer_results = list()

num_labels = 11
sel_model = 'vit'
activation_f = 'softmax'
lr = 5e-4
epochs = 15
prob = 0.5
batch_size = 32
aug = False
focal = False

X = np.array(df['filename'])
y = np.array(df['label'])

f1scoretfa = tfa.metrics.F1Score(num_classes=num_labels, average='macro',threshold=0.5)

In [ ]:
test_f1score = []
overall_test_f1score = []
test_fold = []
test_loss = []

trial = 1

agnostic_datagen = ImageDataGenerator(rescale=1./255)

if aug == True:
    train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.2, height_shift_range=0.2,brightness_range=[0.4,1.5],zoom_range=0.3)

for train_ix, test_ix in skf_outer.split(X,y):

    val_f1score = []
    val_loss = []
    
    train_list = []
    #y_train_list = []
    test_list = []

    for ind in train_ix:
        train_list.append([X[ind],y[ind]])
        #y_train_list.append(y[ind])
    
    for ind in test_ix:
        test_list.append([X[ind],y[ind]])

    X_train = pd.DataFrame(train_list, columns =['filename','label'])
    X_test = pd.DataFrame(test_list, columns =['filename','label'])

    test_generator = agnostic_datagen.flow_from_dataframe(X_test, directory = path_data,
							x_col = "filename", y_col = "label",
                            target_size = (224,224),
                            color_mode='rgb',
                            batch_size= batch_size,
							class_mode = "categorical", shuffle = True)
    
    skf_inner = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)

    nfold = 1

    for train_index, val_index in skf_inner.split(X_train['filename'],X_train['label']):
        
        train_list = []
        valid_list = []

        for ind in train_ix:
            train_list.append([X[ind],y[ind]])
        
        for ind in test_ix:
            valid_list.append([X[ind],y[ind]])

        X_train = pd.DataFrame(train_list, columns =['filename','label'])
        X_valid = pd.DataFrame(valid_list, columns =['filename','label'])


        train_generator = train_datagen.flow_from_dataframe(X_train, directory = path_data,
                                                               x_col = "filename", y_col = "label",
                                                               target_size = (224,224),
                                                               color_mode='rgb',
                                                               batch_size= batch_size,
                                                               class_mode = "categorical", shuffle = True)
        
        validation_generator = agnostic_datagen.flow_from_dataframe(X_valid, directory = path_data,
                                                                 x_col = "filename", y_col = "label",
                                                                 target_size = (224,224),
                                                                 color_mode='rgb',
                                                                 batch_size= batch_size,
                                                                 class_mode = "categorical")
        
        #feature_extractor,vit_model = classification_model(sel_model,lr,activation_f,num_labels,prob,num_neurons)
        model = returnModel(num_labels,activation_f,batch_size)
        #print(model.summary())
        
        #fname = "weights-improvement-{epoch:02d}-{val_f1_score:.2f}.hdf5"
        
        #checkpoint = ModelCheckpoint(fname, monitor='val_f1_score', verbose=1, save_best_only=True, mode='max')

        #early_stopping = EarlyStopping(monitor='loss', patience=5)
        #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=0.000002)
        #callbacklist = [early_stopping,reduce_lr]

        adams_family = Adam(learning_rate=lr,decay=0.01)

        if focal == True:
            loss_function = categorical_focal_loss(gamma=2)
        else:
            loss_function = 'categorical_crossentropy'
        
        model.compile(optimizer = adams_family,
                      loss = loss_function,
                      metrics = [f1scoretfa])

        history_fine = model.fit(train_generator,
                         epochs=epochs,
                         batch_size=batch_size,
                         validation_data=validation_generator)#,
                         #callbacks=callbacklist
                         #)
        
        f1 = history_fine.history['f1_score']
        val_f1 = history_fine.history['val_f1_score']

        loss = history_fine.history['loss']
        val_loss = history_fine.history['val_loss']

        val_f1score.append(np.mean(np.array(val_f1)))
        val_loss.append(np.mean(np.array(val_loss)))

        nfold+=1

        print(f'NFold {nfold} Validation loss: {val_loss} / Validation f-score: {val_f1}')
        #del model

        y_prob = model.predict(test_generator)
        y_pred = np.argmax(y_prob,axis=1)
        y_true = test_generator.classes

        f1score = f1_score(y_true, y_pred, average='macro')
        #score = model.evaluate(test_generator,batch_size=batch_size) 
        #print(f'Trial {trial} Test loss: {score[0]} / Test accuracy: {score[1]}\n')

        trial+=1

        test_f1score.append(f1score)
        #test_loss.append(np.mean(np.array(val_loss)))

        del model
        
    overall_test_f1score.append(np.mean(np.array(test_f1score)))

Found 509 validated image filenames belonging to 11 classes.
Found 2034 validated image filenames belonging to 11 classes.
Found 509 validated image filenames belonging to 11 classes.


All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Epoch 1/15
64/64 [==============================] - 57s 721ms/step - loss: 2.6520 - f1_score: 0.2770 - val_loss: 2.1497 - val_f1_score: 0.3137


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

y_prob = model.predict(test_generator)
y_pred = np.argmax(y_prob,axis=1)

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
z = confusion_matrix(y_true, y_pred)
z

In [ ]:
print(classification_report(y_true, y_pred, target_names=classes))